We will now define RS (regular spiking) excitatory neuron and FS (fast spiking) inhibitory neuron. These 2 types are enough to model part of the cortex in terms of dynamical system.

These parituclar parameters were taken from Izhikevich's [paper](https://www.pnas.org/doi/10.1073/pnas.0712231105) (2008).

In [ ]:
import numpy as np
from scipy.integrate import ode #solving DE
import plotly.subplots as sp #visualizations
import plotly.graph_objs as go #visualizations

#LaTeX workaround
import plotly
from IPython.display import display, HTML
plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))

from timeit import timeit

## Consts

In [ ]:
V_PEAK_PARAM = "v_peak"
A_PARAM = "a"
B_PARAM = "b"
c_PARAM = "c"
D_PARAM = "d"

C_PARAM = "C"
K_PARAM = "k"
V_R_PARAM = "v_r"
V_T_PARAM = "v_t"

TAU_AMPA = "tau_ampa"
TAU_NMDA = "tau_nmda"
TAU_GABAA = "tau_gabaa"
TAU_GABAB = "tau_gabab"

# Utils

In [ ]:
def plot_membrane_voltage(v: np.array, T: np.array):
    """Plots membrane potential progression over the time."""
    fig = go.Figure()

    trace = go.Scatter(x=T, y=v, mode='lines', name=r"$\text{Membrane voltage} \: (\text{in} \: mV)$")
    fig.add_trace(trace)

    fig.update_layout(title='Membrane voltage',
                      xaxis_title=r'$\text{Time} \: (\text{in} \: ms)$',
                      yaxis_title=r'$mV$')
    
    return trace, fig

In [ ]:
def plot_all_membrane_voltage(v: np.array, T: np.array):
    """Plots all membrane potential progression over the time."""
    fig = go.Figure()

    for i in range(v.shape[1]):
        trace = go.Scatter(x=T, y=v[:, i], mode='lines', name=r"$\text{Membrane voltage for neuron }" f"{i}" r"\: (\text{in} \: mV)$")
        fig.add_trace(trace)

    fig.update_layout(title='Membrane voltage',
                      xaxis_title=r'$\text{Time} \: (\text{in} \: ms)$',
                      yaxis_title=r'$mV$')
    
    return trace, fig

In [ ]:
def firing_rates(firings: np.ndarray, T: np.ndarray):
    """Plots all firing times as raster plot."""

    fig = go.Figure()

    firing_indices = np.argwhere(firings == 1)

    trace = go.Scatter(x=firing_indices[:, 0], y=firing_indices[:, 1],
                           mode='markers', marker=dict(color='black', size=4))

    fig.add_trace(trace)

    fig.update_layout(
    xaxis_title='Time Step',
    yaxis_title='Neuron',
    title='Raster Plot',
    yaxis=dict(tickmode='array', tickvals=list(range(firings.shape[1])), ticktext=list(range(1, firings.shape[1] + 1)))
    )


    return trace, fig

In [ ]:
def plot_membrane_voltage_against_recovery(voltage: np.array, recovery_variable: np.array):
    """Plots membrane voltage against recovery variable progression over the time forming dynamical plot.
    x-axis - recovery variable (u)
    y-axis - membrane voltage (v)"""
    fig = go.Figure()

    trace = go.Scatter(x=recovery_variable, y=voltage, mode='lines', name='Membrane voltage (in mV) against recovery variable (u)')

    fig.add_trace(go.Scatter(x=recovery_variable, y=voltage, mode='lines', name='Membrane voltage (in mV) against potassium recovery variable (u)'))

    fig.update_layout(title='Membrane voltage against recovery variable',
                      yaxis_title='mV')
    
    return trace, fig

# Excitatory RS Neuron

In [ ]:
C_RS = 100 #membrane capacitance
a_RS = 0.01 #recovery time constant
b_RS = 5 #determines whether u is an amplifying (b < 0) or resonant (b > 0)
c_RS = -60 #voltage reset value
d_RS = 400 #total amount of outward minus inward currents activated during the spike and affecting the after-spike behavior

v_r_RS = - 60 #resting membrane potential
v_t_RS = - 50 #instantaneous threshold potential
v_peak_RS = 50 #spike cutoff value

k_RS = 3

v_RS = v_r_RS #membrane potential
u_RS = 0 #recovery current

start_state = np.array([v_RS, u_RS])

params_RS = {A_PARAM: a_RS, 
          B_PARAM: b_RS, 
          c_PARAM: c_RS, 
          D_PARAM: d_RS, 
          C_PARAM: C_RS, 
          K_PARAM: k_RS, 
          V_PEAK_PARAM: v_peak_RS, 
          V_R_PARAM: v_r_RS, 
          V_T_PARAM: v_t_RS}

In [ ]:
t_min = 0
t_max = 1000
sim_steps = 10000

T = np.linspace(t_min, t_max, sim_steps)
delta_T = t_max/sim_steps

In [ ]:
def simulation_run(res_state: np.ndarray, I, T: np.ndarray, delta_T: float, params: dict):
    v_curr, u_curr = res_state
    v = [v_curr]
    u = [u_curr]
    for i in range(len(T)):
        if v_curr >= params[V_PEAK_PARAM]:
            v_new = params[c_PARAM]
            u_new = u_curr + params[D_PARAM]
        else:
            v_new = v_curr + delta_T*(params[K_PARAM]*(v_curr - params[V_R_PARAM])*(v_curr - params[V_T_PARAM]) - u_curr + I(T[i]))/params[C_PARAM]
            u_new = u_curr + delta_T*params[A_PARAM]*(params[B_PARAM]*(v_curr - params[V_R_PARAM]) - u_curr)
        
        v_curr = v_new
        u_curr = u_new
        v.append(v_curr)
        u.append(u_curr)
    
    return v, u

## DC Current

In [ ]:
def I_dc(t):
    if t > 100.0:
        return 125.0
    return 0.0

In [ ]:
v, u = simulation_run(start_state, I_dc, T, delta_T, params_RS)

In [ ]:
_, fig = plot_membrane_voltage(v, T)
fig.show()

In [ ]:
_, fig = plot_membrane_voltage_against_recovery(v, u)
fig.show()

# Inhibitory FS Interneuron

In [ ]:
C_FS = 20 #membrane capacitance
a_FS = 0.15 #recovery time constant
b_FS = 8
c_FS = -55 #voltage reset value
d_FS = 200 #total amount of outward minus inward currents activated during the spike and affecting the after-spike behavior

v_r_FS = - 55 #resting membrane potential
v_t_FS = - 40 #instantaneous threshold potential
v_peak_FS = 25 #spike cutoff value

k_FS = 1

v_FS = v_r_FS #membrane potential
u_FS = 0 #recovery current

start_state = np.array([v_FS, u_FS])

params_FS = {A_PARAM: a_FS, 
          B_PARAM: b_FS, 
          c_PARAM: c_FS, 
          D_PARAM: d_FS, 
          C_PARAM: C_FS, 
          K_PARAM: k_FS, 
          V_PEAK_PARAM: v_peak_FS, 
          V_R_PARAM: v_r_FS, 
          V_T_PARAM: v_t_FS}

Wow! Awesome threshold around 70(pA). Neuron fires at constant rate if I_DC value is greater then 73(pA).

In [ ]:
v, u = simulation_run(start_state, I_dc, T, delta_T, params_FS)

In [ ]:
_, fig = plot_membrane_voltage(v, T)
fig.show()

In [ ]:
_, fig = plot_membrane_voltage_against_recovery(v, u)
fig.show()

# Synapse Model

In [ ]:
Ne = 2
Ni = 2

In [ ]:
tau_ampa = 5
tau_nmda = 100
tau_gabaa = 6
tau_gabab = 150

sigma = 14.4
e_to_e = 5
e_to_i = 10
i_to_e = 0
i_to_i = 10

In [ ]:
start_state = np.column_stack((np.append(v_RS*np.ones(Ne), v_FS*np.ones(Ni)), np.append(u_RS*np.ones(Ne), u_FS*np.ones(Ni)))) #matrix (Ne + Ni) x 2 - (v, u) for each neuron

params_network = {A_PARAM: np.append(a_RS*np.ones(Ne), a_FS*np.ones(Ni)), 
          B_PARAM: np.append(b_RS*np.ones(Ne), b_FS*np.ones(Ni)), 
          c_PARAM: np.append(c_RS*np.ones(Ne), c_FS*np.ones(Ni)), 
          D_PARAM: np.append(d_RS*np.ones(Ne), d_FS*np.ones(Ni)), 
          C_PARAM: np.append(C_RS*np.ones(Ne), C_FS*np.ones(Ni)), 
          K_PARAM: np.append(k_RS*np.ones(Ne), k_FS*np.ones(Ni)),
          V_PEAK_PARAM: np.append(v_peak_RS*np.ones(Ne), v_peak_FS*np.ones(Ni)), 
          V_R_PARAM: np.append(v_r_RS*np.ones(Ne), v_r_FS*np.ones(Ni)), 
          V_T_PARAM: np.append(v_t_RS*np.ones(Ne), v_t_FS*np.ones(Ni)),
          TAU_AMPA: tau_ampa*np.ones((Ne + Ni, Ne + Ni)),
          TAU_NMDA: tau_nmda*np.ones((Ne + Ni, Ne + Ni)),
          TAU_GABAA: tau_gabaa*np.ones((Ne + Ni, Ne + Ni)),
          TAU_GABAB: tau_gabab*np.ones((Ne + Ni, Ne + Ni))}

In [ ]:
def run_network_sim(Ne: int, Ni: int, T: np.ndarray, delta_T: float, start_state: np.ndarray, weights: np.ndarray, currents: np.ndarray, params: dict):
    """
    Inputs:
    - Ne (int): number of excitatory neurons.
    - Ni (int): number of inhibitory neurons.
    - T (np.ndarray of float(s)): array of simlutation time points.
    - delta_T (float): integration constant (time step value).
    - start_state (np.ndarray of float(s))): start values of (v, u) for each neuron in network.
    - weights (np.ndarray of float(s)): synaptic weights (i, j) -> weight from i to j.
    - currents (np.ndarray of float(s) or function): injected current to neuron i at time t.
    - params (dict): parameters of neurons DST.

    """
    firings = np.array(np.zeros((len(T) + 1, Ne + Ni)))
    states = np.array(np.zeros((len(T) + 1, Ne + Ni, 2)))
    conductances = np.array(np.zeros((len(T) + 1, Ne + Ni, Ne + Ni, 4))) #4-dimensional because it depends on synaptic weights unfortunately:( will see what we can do with this!
    states[0, :, :] = start_state
    synaptic_input = np.array(np.zeros((len(T) + 1, Ne + Ni)))
    # times = {
    #     "who_fires": 0,
    #     "masks": 0,
    #     "synaptic_input": 0,
    #     "firing_states": 0,
    #     "non-firing_states": 0,
    #     "firing_conductance": 0,
    #     "non-firing_conductance": 0
    # }

    for i in range(1, len(T)):

        ## DECIDE WHO FIRES
        # start = timeit()

        firing_neurons_idx = np.where(states[i - 1, :, 0] >= params[V_PEAK_PARAM])[0] #who is on fire? 🥵

        # end = timeit()
        # times["who_fires"] += end - start
        # start = timeit()

        exc_firing_neurons_idx = firing_neurons_idx[firing_neurons_idx < Ne]
        inh_firing_neurons_idx = firing_neurons_idx[firing_neurons_idx >= Ne]

        mask = np.zeros(Ne + Ni, dtype=bool)
        mask[firing_neurons_idx] = True

        exc_mask = np.zeros(Ne + Ni, dtype=bool)
        exc_mask[exc_firing_neurons_idx] = True

        exc_inv_mask = np.ones(Ne, dtype=bool)
        exc_inv_mask[exc_firing_neurons_idx] = False
        exc_inv_mask = np.append(exc_inv_mask, np.zeros(Ni, dtype = bool))

        inh_mask = np.zeros(Ne + Ni, dtype=bool)
        inh_mask[inh_firing_neurons_idx] = True

        inh_inv_mask = np.ones(Ne + Ni, dtype=bool)
        inh_inv_mask[inh_firing_neurons_idx] = False
        inh_inv_mask[np.arange(Ne)] = False

        # end = timeit()
        # times["masks"] += end - start
        # start = timeit()


        ## UPDATE OF SYNAPTIC INPUT
        synaptic_input[i, :] = np.multiply(np.sum(conductances[i-1, :, :, 0], axis = 0), states[i-1, :, 0] - 0 * np.ones((Ne + Ni))) \
        + np.multiply(np.multiply(np.sum(conductances[i-1, :, :, 1], axis = 0), states[i-1, :, 0] - 0 * np.ones((Ne + Ni))), np.divide(np.divide(np.multiply(states[i-1, :, 0] + 80 * np.ones((Ne + Ni)), states[i-1, :, 0] + 80 * np.ones((Ne + Ni))), np.multiply(60 * np.ones((Ne + Ni)), 60 * np.ones((Ne + Ni)))), np.ones(Ne + Ni) + np.divide(np.multiply(states[i-1, :, 0] + 80 * np.ones((Ne + Ni)), states[i-1, :, 0] + 80 * np.ones((Ne + Ni))), np.multiply(60 * np.ones((Ne + Ni)), 60 * np.ones((Ne + Ni)))))) \
        + np.multiply(np.sum(conductances[i-1, :, :, 2], axis = 0), states[i-1, :, 0] + 70 * np.ones((Ne + Ni))) \
        + np.multiply(np.sum(conductances[i-1, :, :, 3], axis = 0), states[i-1, :, 0] + 90 * np.ones((Ne + Ni)))
        # AMPA + NMDA + GABA A + GABA B

        # end = timeit()
        # times["synaptic_input"] += end - start
        # start = timeit()

        ## UPDATE OF STATES (U, V)

        ## FOR FIRING NEURONS

        states[i, mask, 0] = params[c_PARAM][mask] #update of v for firing neurons
        states[i, mask, 1] = states[i - 1, mask, 1] + params[D_PARAM][mask] #update of u for firing neurons
        firings[i, mask] = 1 #save information who is on fire for raster plot

        # end = timeit()
        # times["firing_states"] += end - start
        # start = timeit()


        ## FOR NON-FIRING NEURONS

        states[i, ~mask, 0] = states[i - 1, ~mask, 0] + delta_T*np.divide((np.multiply(params[K_PARAM][~mask], np.multiply((states[i - 1, ~mask, 0] - params[V_R_PARAM][~mask]), (states[i - 1, ~mask, 0] - params[V_T_PARAM][~mask])))) - states[i - 1, ~mask, 1] + currents[i, ~mask] - synaptic_input[i, ~mask], params[C_PARAM][~mask])  #update v for non-firing neurons: please observe presence of external input as well as internal (synaptic) one
        states[i, ~mask, 1] = states[i - 1, ~mask, 1] + delta_T*np.multiply(params[A_PARAM][~mask], (np.multiply(params[B_PARAM][~mask], (states[i - 1, ~mask, 0] - params[V_R_PARAM][~mask]))) - states[i - 1, ~mask, 1]) #update u for non-firing neurons

        # end = timeit()
        # times["non-firing_states"] += end - start
        # start = timeit()

        ## UPDATE OF CONDUCTANCES AFTER UPDATE OF (V, U) ##

        ## FOR FIRING NEURONS

        #for excitatory neurons only AMPA & NMDA should change
        conductances[i, exc_mask, :, 0] = conductances[i - 1, exc_mask, :, 0] + weights[exc_mask, :]
        conductances[i, exc_mask, :, 1] = conductances[i - 1, exc_mask, :, 1] + weights[exc_mask, :]

        #for inhibitory neurons only GABA (A & B) should change 
        conductances[i, inh_mask, :, 2] = conductances[i - 1, inh_mask, :, 2] + weights[inh_mask, :]
        conductances[i, inh_mask, :, 3] = conductances[i - 1, inh_mask, :, 3] + weights[inh_mask, :]

        # end = timeit()
        # times["firing_conductance"] += end - start
        # start = timeit()

        ## FOR NON-FIRING NEURONS

        #for excitatory neurons only AMPA & NMDA should change
        conductances[i, exc_inv_mask, :, 0] = conductances[i - 1, exc_inv_mask, :, 0] - delta_T*np.divide(conductances[i - 1, exc_inv_mask, :, 0], params[TAU_AMPA][exc_inv_mask])
        conductances[i, exc_inv_mask, :, 1] = conductances[i - 1, exc_inv_mask, :, 1] - delta_T*np.divide(conductances[i - 1, exc_inv_mask, :, 1], params[TAU_NMDA][exc_inv_mask])
        #for inhibitory neurons only GABA (A & B) should change 
        conductances[i, inh_inv_mask, :, 2] = conductances[i - 1, inh_inv_mask, :, 2] - delta_T*np.divide(conductances[i - 1, inh_inv_mask, :, 2], params[TAU_GABAA][inh_inv_mask])
        conductances[i, inh_inv_mask, :, 3] = conductances[i - 1, inh_inv_mask, :, 3] - delta_T*np.divide(conductances[i - 1, inh_inv_mask, :, 3], params[TAU_GABAB][inh_inv_mask])

        # end = timeit()
        # times["non-firing_conductance"] += end - start

        # print(times)
    
    return states, firings, conductances

In [ ]:
weights = np.array(np.zeros((Ne + Ni, Ne + Ni))) #matrix (Ne + Ni) x (Ne + Ni) (i, j: i -> j), thus sum by column for input of j
        
weights[0, 2] = 5
weights[1, 3] = 5
weights[2, 1] = 10
weights[3, 0] = 10

currents = np.array(np.zeros((len(T), Ne + Ni)))
currents[:, 0] = 1250
currents[:, 1] = 1200

In [ ]:
states, firings, conductances = run_network_sim(Ne, Ni, T, delta_T, start_state, weights, currents, params_network)

In [ ]:
_, fig = plot_all_membrane_voltage(states[:, :, 0], T)
fig.show()

In [ ]:
_, fig = firing_rates(firings, T)
fig.show()

In [ ]:
np.sum(firings, axis = 0)

# Cano-Colino et al. Modeling

For 2 neurons.

In [ ]:
weights = np.array(np.zeros((Ne + Ni, Ne + Ni))) #matrix (Ne + Ni) x (Ne + Ni) (i, j: i -> j), thus sum by column for input of j

for i in range(Ne):
    for j in range(Ne):
        if i != j:
            angle_i = i / Ne * 360
            angle_j = j / Ne * 360
            weights[i, j] = e_to_e * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/sigma**2)

for i in range(Ne):
    for j in range(Ni):
        angle_i = i / Ne * 360
        angle_j = j / Ni * 360
        weights[i, Ne + j] = e_to_i * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/sigma**2)

for i in range(Ni):
    for j in range(Ne):
        angle_i = i / Ni * 360
        angle_j = j / Ne * 360
        weights[Ne + i, j] = i_to_e * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/sigma**2)

for i in range(Ni):
    for j in range(Ni):
        if i != j:
            angle_i = i / Ni * 360
            angle_j = j / Ni * 360
            weights[Ne + i, Ne + j] = i_to_i * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/sigma**2)

currents = np.array(np.zeros((len(T), Ne + Ni)))
currents[:, 0] = 500
currents[:, 1] = 400

In [ ]:
states, firings, conductances = run_network_sim(Ne, Ni, T, delta_T, start_state, weights, currents, params_network)

In [ ]:
_, fig = plot_all_membrane_voltage(states[:, :, 0], T)
fig.show()

In [ ]:
_, fig = firing_rates(firings, T)
fig.show()

In [ ]:
np.sum(firings, axis = 0)

For 100 excitatory and 20 inhibitory neurons.

In [ ]:
Ne = 100
Ni = 20

In [ ]:
start_state = np.column_stack((np.append(v_RS*np.ones(Ne), v_FS*np.ones(Ni)), np.append(u_RS*np.ones(Ne), u_FS*np.ones(Ni)))) #matrix (Ne + Ni) x 2 - (v, u) for each neuron

params_network = {A_PARAM: np.append(a_RS*np.ones(Ne), a_FS*np.ones(Ni)), 
          B_PARAM: np.append(b_RS*np.ones(Ne), b_FS*np.ones(Ni)), 
          c_PARAM: np.append(c_RS*np.ones(Ne), c_FS*np.ones(Ni)), 
          D_PARAM: np.append(d_RS*np.ones(Ne), d_FS*np.ones(Ni)), 
          C_PARAM: np.append(C_RS*np.ones(Ne), C_FS*np.ones(Ni)), 
          K_PARAM: np.append(k_RS*np.ones(Ne), k_FS*np.ones(Ni)),
          V_PEAK_PARAM: np.append(v_peak_RS*np.ones(Ne), v_peak_FS*np.ones(Ni)), 
          V_R_PARAM: np.append(v_r_RS*np.ones(Ne), v_r_FS*np.ones(Ni)), 
          V_T_PARAM: np.append(v_t_RS*np.ones(Ne), v_t_FS*np.ones(Ni)),
          TAU_AMPA: tau_ampa*np.ones((Ne + Ni, Ne + Ni)),
          TAU_NMDA: tau_nmda*np.ones((Ne + Ni, Ne + Ni)),
          TAU_GABAA: tau_gabaa*np.ones((Ne + Ni, Ne + Ni)),
          TAU_GABAB: tau_gabab*np.ones((Ne + Ni, Ne + Ni))}

In [ ]:
weights = np.array(np.zeros((Ne + Ni, Ne + Ni))) #matrix (Ne + Ni) x (Ne + Ni) (i, j: i -> j), thus sum by column for input of j

for i in range(Ne):
    for j in range(Ne):
        if i != j:
            angle_i = i / Ne * 360
            angle_j = j / Ne * 360
            weights[i, j] = e_to_e * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/sigma**2)

for i in range(Ne):
    for j in range(Ni):
        angle_i = i / Ne * 360
        angle_j = j / Ni * 360
        weights[i, Ne + j] = e_to_i * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/sigma**2)

for i in range(Ni):
    for j in range(Ne):
        angle_i = i / Ni * 360
        angle_j = j / Ne * 360
        weights[Ne + i, j] = i_to_e * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/sigma**2)

for i in range(Ni):
    for j in range(Ni):
        if i != j:
            angle_i = i / Ni * 360
            angle_j = j / Ni * 360
            weights[Ne + i, Ne + j] = i_to_i * np.exp(-(min(max(angle_i, angle_j) - min(angle_i, angle_j), 360 - (max(angle_i, angle_j) - min(angle_i, angle_j))))**2/sigma**2)

currents = np.array(np.zeros((len(T), Ne + Ni)))
currents[:, 0] = 500
currents[:, 1] = 400

In [ ]:
states, firings, conductances = run_network_sim(Ne, Ni, T, delta_T, start_state, weights, currents, params_network)

In [ ]:
_, fig = plot_all_membrane_voltage(states[:, :, 0], T)
fig.show()

In [ ]:
_, fig = firing_rates(firings, T)
fig.show()

In [ ]:
np.sum(firings, axis = 0)